In [ ]:
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Define your stations, corresponding node numbers, and custom names
stations_and_nodes = [
    {'station': '8632200', 'node': 104845, 'name': 'Kiptopeke, VA'},  # COORD: 37.1653, -75.9883
    {'station': '8571892', 'node': 12948, 'name': 'Cambridge, MD'},  # COORD: 38.5717, -76.0683
    {'station': '8516945', 'node': 52435, 'name': 'Kings Point, NY'},  # COORD: 40.8100, -73.7650
    {'station': '8722956', 'node': 76314, 'name': 'South Port, FL'},  # COORD: 26.0817, -80.1167
    {'station': '8720218', 'node': 90753, 'name': 'Mayport, FL'},  # COORD: 30.3917, -81.3983
]

# Load netCDF files
withoutoffset = netCDF4.Dataset(r'/work2/07174/soelem/validation/base_run/fort.63.nc', 'r')
withoffset = netCDF4.Dataset(r'/work2/07174/soelem/validation/combined/fort.63.nc', 'r')

# Get the minimum length of time dimension from both datasets to synchronize their lengths
min_time_length = min(withoutoffset.variables['zeta'].shape[0], withoffset.variables['zeta'].shape[0])

# Setup the figure for subplots with adjusted figsize for wider and shorter appearance
fig, axs = plt.subplots(len(stations_and_nodes), 1, figsize=(26, 10), dpi=600, sharex=True)

for i, station_info in enumerate(stations_and_nodes):
    node_index = station_info['node']

    # Check if the node index is within the bounds for the 'zeta' variable in both datasets
    if node_index < withoutoffset.variables['zeta'].shape[1] and node_index < withoffset.variables['zeta'].shape[1]:
        # Fetch data only up to the synchronized time length
        withoutoffset_data = withoutoffset.variables['zeta'][:min_time_length, node_index]
        withoffset_data = withoffset.variables['zeta'][:min_time_length, node_index]
        
        # Calculate the difference
        bias = withoffset_data - withoutoffset_data
        
        # Convert to DataFrame
        df_bias = pd.DataFrame(bias, columns=['Bias'])
        
        # Create date range for index
        date_range = pd.date_range(start='2023-07-16 00:00:00', periods=min_time_length, freq='60min')
        df_bias['Date Time'] = date_range
        
        # Plotting on subplot
        axs[i].plot(df_bias['Date Time'], df_bias['Bias'], linestyle='-', linewidth=5, color='red', label='Bias')
        axs[i].set_ylabel(f'{station_info["name"]}', fontsize=13)
        axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%d-%b'))  # Set date format
        axs[i].tick_params(axis='x', labelrotation=45, labelsize=20)  # Increase x-tick font size
        axs[i].tick_params(axis='y', labelsize=15)  # Increase y-tick font size
        axs[i].grid(True)
        axs[i].set_facecolor('#E6E6FA')  # Light blue background for each subplot
    else:
        print(f"Node index {node_index} for {station_info['name']} is out of bounds.")

# Add the legend to the last subplot with adjustments for size and gap
axs[-1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.75), fancybox=True, shadow=True, ncol=1, fontsize=30)

plt.tight_layout()
plt.show()